In [38]:
'''
file_id_mesgs: general info about Garmin device and activity,
file_creator_mesgs: software version,
event_mesgs: general info about event? (ex. alert for Garmin live track?),
device_info_mesgs: info of all devices and sensors owned,
device_settings_mesgs: time and specific device settings,
user_profile_mesgs: user profile info and settings,
sport_mesgs: activity sport,
zones_target_mesgs: HR and Power tresholds,

record_mesgs: all activity info!!!,

lap_mesgs: lap data,
time_in_zone_mesgs: HR and Power zones spent during the activity,
session_mesgs: training session info recap,
activity_mesgs: activity brief recap
'''

from garmin_fit_sdk import Decoder, Stream
import folium
import dash
import math
from dash import html

mapFileName = "route_map.html"

class Position:
    SEMICIRCLE_TO_DEGREE = 180 / (2**31)

    def __init__(self, latitude: float, longitude: float):
        self.latitude = self.semicircles_to_degrees(latitude)
        self.longitude = self.semicircles_to_degrees(longitude)

    @staticmethod
    def semicircles_to_degrees(value: float) -> float:
        return value * Position.SEMICIRCLE_TO_DEGREE

    def __repr__(self):
        return f"Position(latitude={self.latitude}, longitude={self.longitude})"


def find_center(coordinates):
    if not coordinates:
        return None
    x_center = sum(x for x, y in coordinates) / len(coordinates)
    y_center = sum(y for x, y in coordinates) / len(coordinates)
    return (x_center, y_center)


def parse_fit_file(filepath):
    stream = Stream.from_file(filepath)
    decoder = Decoder(stream)
    messages, errors = decoder.read()
    if errors:
        print(f"Errors found while decoding FIT file: {errors}")
    return messages


def extract_coordinates(record_messages):
    coordinates = []
    position_lat = "position_lat"
    position_long = "position_long"
    for message in record_messages:
        if position_lat in message and position_long in message:
            coordinates.append(
                Position(message[position_lat], message[position_long])
            )
    return coordinates


def create_and_save_route_map(coordinates, output_file="map.html"):
    folium_coordinates = [[coord.latitude, coord.longitude] for coord in coordinates]
    map_center = find_center(folium_coordinates)
    start = (coordinates[0].latitude, coordinates[0].longitude)
    end = (coordinates[-1].latitude, coordinates[-1].longitude)
    m = folium.Map(location=map_center, zoom_start=11)
    folium.Marker(
        location=start,
        tooltip="Start",
        popup="Home sweet home",
        icon=folium.Icon(color="green")
    ).add_to(m)
    folium.Marker(
        location=end,
        tooltip="End",
        popup="Home sweet home",
        icon=folium.Icon(color="red")
    ).add_to(m)
    folium.PolyLine(folium_coordinates, tooltip="route").add_to(m)
    m.save(output_file)
    print(f"Map saved to {output_file}")

def showDashboard(map_path):
    # Inizializzazione dell'app Dash
    app = dash.Dash(__name__)
    
    # Layout dell'applicazione
    app.layout = html.Div([
        html.H1("Overview"),
        html.Iframe(
            srcDoc=open(mapFileName, "r").read(),
            width="100%",
            height="600px"
        )
    ])

    if __name__ == '__main__':
        app.run_server(debug=True)

def get_activity_duration(total_seconds):
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return hours, minutes, seconds

def get_activity_duration__as_string(total_seconds):
    hours, minutes, seconds = get_activity_duration(total_seconds)
    is_over_10h_string = "0" if hours < 10 else ""
    is_over_10m_string = "0" if minutes < 10 else ""
    return str(fill_zeros(hours)) + ":" + str(fill_zeros(minutes)) + ":" + str(fill_zeros(seconds))

def fill_zeros(param):
    return "0" + str(param) if param < 10 else param

def haversine(coord1, coord2):
    R = 6371  # Raggio della Terra in km
    lat1, lon1 = coord1.latitude, coord1.longitude
    lat2, lon2 = coord2.latitude, coord2.longitude

    # Conversione da gradi a radianti
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calcolo delle differenze
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1

    # Formula dell'Haversine
    a = math.sin(delta_lat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c  # Distanza in km

    return distance

def total_distance(positions):
    total_dist = 0.0
    for i in range(len(positions) - 1):
        total_dist += haversine(positions[i], positions[i + 1])
    return round(total_dist, 2)

def get_positive_ascent(activity_data, altitude_key):
    positive_height_difference = 0

    for i in range(len(activity_data) - 1):
        if(altitude_key in activity_data[i] and altitude_key in activity_data[i+1]):
            current_altitude = activity_data[i][altitude_key]
            future_altitude = activity_data[i + 1][altitude_key]            
            if future_altitude > current_altitude:
                positive_height_difference += (future_altitude - current_altitude)        

    return round(positive_height_difference, 2)

def get_average_parameter(activity_data, parameter_key):
    cumulative_parameter = 0
    valid_parameter_record_counter = 0
    
    for i in range (len(activity_data)):
        if(parameter_key in activity_data[i]):
            valid_parameter_record_counter += 1
            cumulative_parameter += activity_data[i][parameter_key]

    return 0 if valid_parameter_record_counter == 0 else round(cumulative_parameter/valid_parameter_record_counter, 2)    

def get_max_parameter(activity_data, parameter_key):
    max_parameter = -float('inf')
        
    for i in range (len(activity_data)):
        if(parameter_key in activity_data[i]):
            if(max_parameter < activity_data[i][parameter_key]):
                max_parameter = activity_data[i][parameter_key]

    return max_parameter

def get_max_parameter_in_minutes(activity_data, parameter_key, interval_in_minutes):
    start_time_in_seconds = 0
    end_time_in_seconds = 0
    max_parameter_in_minutes = 0
    interval_in_seconds = interval_in_minutes * 60

    for i in range (len(activity_data) - interval_in_seconds):
        if(parameter_key in activity_data[i]):
            average_value = sum(element[parameter_key] if parameter_key in element else 0 for element in activity_data[i:interval_in_seconds + i]) / interval_in_seconds
            if average_value > max_parameter_in_minutes:                
                max_parameter_in_minutes = average_value
                start_time_in_seconds = i
                end_time_in_seconds = i + interval_in_seconds

    start_time = get_activity_duration__as_string(start_time_in_seconds)
    end_time_in_seconds = get_activity_duration__as_string(end_time_in_seconds)
    return round(max_parameter_in_minutes, 2), start_time, end_time_in_seconds


def main():
    filepath = '../data/road_cycling_activity.fit'
    messages = parse_fit_file(filepath)
    record_mesgs = messages["record_mesgs"]
    activity_duration_seconds = len(record_mesgs)
    coordinates = extract_coordinates(record_mesgs)
    duration_hh_mm_ss = get_activity_duration__as_string(activity_duration_seconds)    
    distance = total_distance(coordinates)
    average_speed_km_h = round(distance / (activity_duration_seconds / 3600), 2)
    total_positive_height = get_positive_ascent(record_mesgs, "altitude")    
    average_cadence = get_average_parameter(record_mesgs, "cadence")
    average_heart_rate = get_average_parameter(record_mesgs, "heart_rate")
    average_power = get_average_parameter(record_mesgs, "power")
    max_positive_height = get_max_parameter(record_mesgs, "altitude")    
    max_cadence = get_max_parameter(record_mesgs, "cadence")
    max_heart_rate = get_max_parameter(record_mesgs, "heart_rate")
    max_20_min_heart_rate = get_max_parameter_in_minutes(record_mesgs, "heart_rate", 20)
    max_20_min_power = get_max_parameter_in_minutes(record_mesgs, "power", 20)
    max_20_min_cadence = get_max_parameter_in_minutes(record_mesgs, "cadence", 20)
    max_20_min_speed_m_s, max_20_min_speed_start_time, max_20_min_speed_end_time = get_max_parameter_in_minutes(record_mesgs, "speed", 20)
    max_20_min_speed_km_h = max_20_min_speed_m_s * 3.6
    max_20_min_speed = (max_20_min_speed_km_h, max_20_min_speed_start_time, max_20_min_speed_end_time)
    max_power = get_max_parameter(record_mesgs, "power")
    total_kilojoule = round(average_power * activity_duration_seconds / 1000, 2)
    print(max_20_min_power)
    print(max_20_min_heart_rate)
    print(max_20_min_cadence) 
    print(max_20_min_speed)
    #create_and_save_route_map(coordinates, mapFileName)
    #showDashboard(mapFileName)

main()


(206.86, '01:19:15', '01:39:15')
(168.29, '01:19:41', '01:39:41')
(86.44, '00:36:22', '00:56:22')
(33.624, '01:41:52', '02:01:52')
